# **Azure ADLS GEN2**

In [0]:
spark.conf.set(
  "fs.azure.account.key.tnelectionanalytics.dfs.core.windows.net",
  "********K0YWjaEyvFF+Sw9/wd3v1f0f1ZGXQU5tw7PoyoLuH+J/4lnvRh/DHwANAv09*********=="
)


In [0]:
configs = {
  "fs.azure.account.key.tnelectionanalytics.blob.core.windows.net":
  "********K0YWjaEyvFF+Sw9/wd3v1f0f1ZGXQU5tw7PoyoLuH+J/4lnvRh/DHwANAv09*********=="
}

**Mounting the folders**

In [0]:
dbutils.fs.mount(
  source = "wasbs://raw@tnelectionanalytics.blob.core.windows.net/",
  mount_point = "/mnt/elections_raw",
  extra_configs = configs
)

True

In [0]:
display(dbutils.fs.ls("/mnt/elections_raw"))

path,name,size,modificationTime
dbfs:/mnt/elections_raw/elections/,elections/,0,0


In [0]:
dbutils.fs.mount(
  source = "wasbs://curated@tnelectionanalytics.blob.core.windows.net/",
  mount_point = "/mnt/elections_curated",
  extra_configs = configs
)

True

In [0]:
# Check the root of the DBFS
dbutils.fs.ls("/mnt/") 



[FileInfo(path='dbfs:/mnt/elections_curated/', name='elections_curated/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/elections_raw/', name='elections_raw/', size=0, modificationTime=0)]

In [0]:
%fs ls /mnt/elections_raw/elections/

path,name,size,modificationTime
dbfs:/mnt/elections_raw/elections/1971.csv,1971.csv,16902,1765314908000
dbfs:/mnt/elections_raw/elections/1977.csv,1977.csv,25725,1765314908000
dbfs:/mnt/elections_raw/elections/1980.csv,1980.csv,26592,1765314909000
dbfs:/mnt/elections_raw/elections/1984.csv,1984.csv,26426,1765314909000
dbfs:/mnt/elections_raw/elections/1989.csv,1989.csv,25209,1765314909000
dbfs:/mnt/elections_raw/elections/1991.csv,1991.csv,26775,1765314909000
dbfs:/mnt/elections_raw/elections/1996.csv,1996.csv,28353,1765314909000
dbfs:/mnt/elections_raw/elections/2001.csv,2001.csv,28772,1765314909000
dbfs:/mnt/elections_raw/elections/2006.csv,2006.csv,27330,1765314909000
dbfs:/mnt/elections_raw/elections/2011.csv,2011.csv,32222,1765314909000


**Cleaning the Raw to bronze data**

In [0]:
files = [
    "dbfs:/mnt/elections_raw/elections/1971.csv",
    "dbfs:/mnt/elections_raw/elections/1977.csv",
    "dbfs:/mnt/elections_raw/elections/1980.csv",
    "dbfs:/mnt/elections_raw/elections/1984.csv",
    "dbfs:/mnt/elections_raw/elections/1989.csv",
    "dbfs:/mnt/elections_raw/elections/1991.csv",
    "dbfs:/mnt/elections_raw/elections/1996.csv",
    "dbfs:/mnt/elections_raw/elections/2001.csv",
    "dbfs:/mnt/elections_raw/elections/2006.csv"
]

from pyspark.sql.types import *
from pyspark.sql.functions import regexp_replace, col, lit, regexp_extract, round

# Define schema
election_schema = StructType([
    StructField("ac_name", StringType(), True),
    StructField("ac_no", LongType(), True),
    StructField("type", StringType(), True),
    StructField("winning_cand", StringType(), True),
    StructField("party", StringType(), True),
    StructField("totelectors", StringType(), True),
    StructField("totvotes", StringType(), True),
    StructField("poll_percentage", StringType(), True),
    StructField("margin", StringType(), True),
    StructField("winning_percentage", StringType(), True)
])

df = spark.read.option("header", True).schema(election_schema).csv(files)

#remove records when all the columns are empty
df = df.na.drop(how='all')

df = (
    df.withColumn("totelectors", regexp_replace(col("totelectors"), ",", "").cast("long"))
       .withColumn("totvotes", regexp_replace(col("totvotes"), ",", "").cast("long"))
       .withColumn("margin", regexp_replace(col("margin"), ",", "").cast("long"))
       .withColumn("poll_percentage", round(regexp_replace(col("poll_percentage"), "%", "").cast("float")).cast("long"))
       .withColumn("winning_percentage", round(regexp_replace(col("winning_percentage"), "%", "").cast("float")).cast("long"))
)

# Insert 'district' column between 'type' and 'winning_cand'
cols = df.columns
insert_at = cols.index("type") + 1
new_cols = cols[:insert_at] + ["district"] + cols[insert_at:]
df = df.withColumn("district", lit(None).cast(StringType())).select(*new_cols)

# Add filename (year) as int as the first column, extracting 4 digits before .csv
df = df.withColumn(
    "filename",
    regexp_extract(col("_metadata.file_name"), r"(\d{4})\.csv$", 1).cast("int")
).select("filename", *df.columns)

display(df.limit(10))

filename,ac_name,ac_no,type,district,winning_cand,party,totelectors,totvotes,poll_percentage,margin,winning_percentage
2001,Royapuram,1,GEN,null,Jayakumar.D,All India Anna Dravida Munnetra Kazhagam,156528,78338,50,13712,18
2001,Harbour,2,GEN,null,Anbazhagan.K,Dravida Munetra Kazhagam,129157,51563,40,336,1
2001,Dr. Radhakrishnan Nagar,3,GEN,null,Sekar Babu.P.K,All India Anna Dravida Munnetra Kazhagam,272666,128175,47,27332,21
2001,Park Town,4,GEN,null,Vinayakamurthi S.G,Tamil Maanila Congress (MOOPANAR),140145,64268,46,6377,10
2001,Perambur,5,SC,null,Mahendran K.,Communist Party Of India (MARXIST),276886,132799,48,17223,13
2001,Purasawalkam,6,GEN,null,Ranganathan B,Dravida Munetra Kazhagam,307376,143574,47,3802,3
2001,Egmore,7,SC,null,Parithi.Ellamvazhuthi,Dravida Munetra Kazhagam,155310,69590,45,86,0
2001,Anna Nagar,8,GEN,null,Arcot Veeraswami .N,Dravida Munetra Kazhagam,376745,160469,43,5578,4
2001,Theagaraya Nagar,9,GEN,null,Anbazhagan.J,Dravida Munetra Kazhagam,277999,119216,43,2499,2
2001,Thousand Lights,10,GEN,null,Stalin M.K.,Dravida Munetra Kazhagam,210474,95428,45,7274,8


In [0]:
files = [
    "dbfs:/mnt/elections_raw/elections/2011.csv",
    "dbfs:/mnt/elections_raw/elections/2016.csv",
    "dbfs:/mnt/elections_raw/elections/2021.csv"
]

from pyspark.sql.types import *
from pyspark.sql.functions import regexp_replace, col, lit, regexp_extract, round

# Define schema
election_schema = StructType([
    StructField("ac_name", StringType(), True),
    StructField("ac_no", LongType(), True),
    StructField("type", StringType(), True),
    StructField("district", StringType(), True),
    StructField("winning_cand", StringType(), True),
    StructField("party", StringType(), True),
    StructField("totelectors", StringType(), True),
    StructField("totvotes", StringType(), True),
    StructField("poll_percentage", StringType(), True),
    StructField("margin", StringType(), True),
    StructField("winning_percentage", StringType(), True)
])

df1 = spark.read.option("header", True).schema(election_schema).csv(files)

#remove records when all the columns are empty
df1 = df1.na.drop(how='all')

df1 = (
    df1.withColumn("totelectors", regexp_replace(col("totelectors"), ",", "").cast("long"))
       .withColumn("totvotes", regexp_replace(col("totvotes"), ",", "").cast("long"))
       .withColumn("margin", regexp_replace(col("margin"), ",", "").cast("long"))
       .withColumn("poll_percentage", round(regexp_replace(col("poll_percentage"), "%", "").cast("float")).cast("long"))
       .withColumn("winning_percentage", round(regexp_replace(col("winning_percentage"), "%", "").cast("float")).cast("long"))
)

# Add filename as int as the first column
df1 = df1.withColumn(
    "filename",
    regexp_extract(col("_metadata.file_name"), r"(\d{4})\.csv$", 1).cast("int")
).select("filename", *df1.columns)

display(df1.limit(10))

filename,ac_name,ac_no,type,district,winning_cand,party,totelectors,totvotes,poll_percentage,margin,winning_percentage
2011,Gummidipoondi,1,GEN,Thiruvallur,Sekar C H,Desiya Murpokku Dravida Kazhagam,215430,178276,83,29101,16
2011,Ponneri,2,SC,Thiruvallur,Pon. Raja,All India Anna Dravida Munnetra Kazhagam,202421,163088,81,31073,19
2011,Tiruttani,3,GEN,Thiruvallur,Arunsubramanian. M,Desiya Murpokku Dravida Kazhagam,236718,191237,81,23930,13
2011,Thiruvallur,4,GEN,Thiruvallur,Ramana B.V,All India Anna Dravida Munnetra Kazhagam,207922,170115,82,23648,14
2011,Poonmallae,5,SC,Thiruvallur,Manimaran R,All India Anna Dravida Munnetra Kazhagam,229436,181531,79,41419,23
2011,Avadi,6,GEN,Thiruvallur,Abdul Rahim.S,All India Anna Dravida Munnetra Kazhagam,277856,199538,72,43238,22
2011,Maduravoyal,7,GEN,Thiruvallur,Beem Rao. G,Communist Party Of India (MARXIST),269487,185925,69,24011,13
2011,Ambattur,8,GEN,Thiruvallur,Vedhachalam. S,All India Anna Dravida Munnetra Kazhagam,264960,186347,70,22717,12
2011,Madavaram,9,GEN,Thiruvallur,Moorthy V,All India Anna Dravida Munnetra Kazhagam,275590,207325,75,34765,17
2011,Thiruvottiyur,10,GEN,Thiruvallur,"Kuppan, K.",All India Anna Dravida Munnetra Kazhagam,221456,164716,74,27291,17


In [0]:
df.write.mode("overwrite").parquet("/mnt/elections_curated/elections/")
df1.write.mode("append").parquet("/mnt/elections_curated/elections/")

In [0]:
df_new = spark.read.parquet("/mnt/elections_curated/elections/")
display(df_new.limit(10))

filename,ac_name,ac_no,type,district,winning_cand,party,totelectors,totvotes,poll_percentage,margin,winning_percentage
1977,Royapuram,1,GEN,null,P. Ponnurangam,Dravida Munetra Kazhagam,141692,72868,51,1591,2
1977,Harbour,2,GEN,null,A. Selvarajan,Dravida Munetra Kazhagam,150577,65557,44,5983,9
1977,Dr. Radhakrishnan Nagar,3,GEN,null,Isari Velan,All India Anna Dravida Munnetra Kazhagam,178751,80748,45,1488,2
1977,Park Town,4,GEN,null,T.S. Nallathambi,Dravida Munetra Kazhagam,145578,69199,48,3226,5
1977,Perambur,5,SC,null,S. Balan,Dravida Munetra Kazhagam,184688,80705,44,13468,17
1977,Purasawalkam,6,GEN,null,K. Anbazhagan,Dravida Munetra Kazhagam,203098,91943,45,19069,21
1977,Egmore,7,SC,null,S.Manimudi,Dravida Munetra Kazhagam,141812,69887,49,7332,11
1977,Anna Nagar,8,GEN,null,M. Karunanidhi,Dravida Munetra Kazhagam,178266,86717,49,16438,19
1977,Theagaraya Nagar,9,GEN,null,R.E. Chandran Jayapal,Dravida Munetra Kazhagam,180871,76185,42,1030,1
1977,Thousand Lights,10,GEN,null,S.J. Sadiq Pasha,Dravida Munetra Kazhagam,167463,72284,43,4858,7


**Cleaning the Bronze data using SQL**

In [0]:
%sql
--CREATE DATABASE TNElection;
SHOW DATABASES;


databaseName
default
tnelection


In [0]:
%sql
use tnelection;
show tables;

database,tableName,isTemporary
,_sqldf,true
,results,true


In [0]:
df_new.write.format("delta").mode("overwrite").saveAsTable("tnelection.results")

In [0]:
%sql SHOW TABLE EXTENDED FROM tnelection LIKE 'results'

database,tableName,isTemporary,information
tnelection,results,false,Catalog: spark_catalog Database: tnelection Table: results Owner: root Created Time: Wed Dec 10 03:07:13 UTC 2025 Last Access: UNKNOWN Created By: Spark 4.0.0 Type: MANAGED Provider: delta Table Properties: [delta.enableDeletionVectors=true] Location: dbfs:/user/hive/warehouse/tnelection.db/results Serde Library: org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe InputFormat: org.apache.hadoop.mapred.SequenceFileInputFormat OutputFormat: org.apache.hadoop.hive.ql.io.HiveSequenceFileOutputFormat Partition Provider: Catalog Schema: root |-- filename: integer (nullable = true) |-- ac_name: string (nullable = true) |-- ac_no: long (nullable = true) |-- type: string (nullable = true) |-- district: string (nullable = true) |-- winning_cand: string (nullable = true) |-- party: string (nullable = true) |-- totelectors: long (nullable = true) |-- totvotes: long (nullable = true) |-- poll_percentage: long (nullable = true) |-- margin: long (nullable = true) |-- winning_percentage: long (nullable = true)
,results,true,Table: results Created Time: Wed Dec 10 02:39:35 UTC 2025 Last Access: UNKNOWN Created By: Spark Type: VIEW View Schema Mode: BINDING Schema: root |-- filename: integer (nullable = true) |-- ac_name: string (nullable = true) |-- ac_no: long (nullable = true) |-- type: string (nullable = true) |-- district: string (nullable = true) |-- winning_cand: string (nullable = true) |-- party: string (nullable = true) |-- totelectors: long (nullable = true) |-- totvotes: long (nullable = true) |-- poll_percentage: long (nullable = true) |-- margin: long (nullable = true) |-- winning_percentage: long (nullable = true)


In [0]:
%sql
UPDATE tnelection.results
SET party = 'Communist Party Of India'
WHERE party = 'CPI'

num_affected_rows
8


In [0]:
%sql
UPDATE tnelection.results
SET party = 'All India Forward Bloc'
WHERE party = 'AIFB'

num_affected_rows
7


In [0]:
%sql
UPDATE tnelection.results
SET party = 'Communist Party Of India (MARXIST)'
WHERE party = 'Communist Party Of India (Marxist)';

UPDATE tnelection.results
SET party = 'Dravida Munetra Kazhagam'
WHERE party = 'DMK';

UPDATE tnelection.results
SET party = 'Indian National Congress'
WHERE party = 'INC';

UPDATE tnelection.results
SET party = 'Indian Union Muslim League'
WHERE party = 'IUML';

UPDATE tnelection.results
SET party = 'Janata Party (JP)'
WHERE party = 'Janta Party';

num_affected_rows
17


In [0]:
%sql
select distinct party from tnelection.results

party
Thayaga Marumalrchi Kazhagam
All India Forward Bloc
Ambedkar Kranti Dal
Jharkhand Party
Pattali Makkal Katchi
Indian National Congress
Gandhi Kamraj National Congress
Puthiya Tamilagam
PSP
M.G.R.Anna D.M.Kazhagam


In [0]:
%sql
select filename as year, ac_name, ac_no, type, district, winning_cand, party, totelectors, totvotes, poll_percentage,
margin, winning_percentage
from tnelection.results
where type is null
order by filename
limit 10

year,ac_name,ac_no,type,district,winning_cand,party,totelectors,totvotes,poll_percentage,margin,winning_percentage
1971,Madurai West,138,null,null,K. T. K. Thangamani,Communist Party Of India,85420,40899,48,44521,52
1971,Thiruthuraipoondi,184,null,null,Manali Kandasami C.,Communist Party Of India,64790,40714,63,24076,37
1971,Perur,104,null,null,N. Marudachalam,Communist Party Of India,68083,39270,58,28813,42
1971,Sirkazhi,175,null,null,Vadivel S.,Communist Party Of India,60051,31977,53,28074,47
1971,Kottur,185,null,null,Subbiah A. K.,Communist Party Of India,64727,47419,73,17308,27
1971,Perundurai,118,null,null,N. K. Palanisamy,Communist Party Of India,68976,38882,56,30094,44
1971,Sedapatti,132,null,null,Thavamani Thevar V.,All India Forward Bloc,55466,20334,37,35132,63
1971,Kovilpatti,210,null,null,S. Alagarsamy,Communist Party Of India,62490,38844,62,23646,38
1971,Rajapalayam,208,null,null,K. Suppu,Communist Party Of India,72997,36827,50,36170,50
1971,Tirumangalam,133,null,null,M. C. A. Rethinasamy Thevar,All India Forward Bloc,64013,36468,57,27545,43


**Mounting Silver container**

In [0]:
dbutils.fs.mount(
  source = "wasbs://analytics@tnelectionanalytics.blob.core.windows.net/",
  mount_point = "/mnt/elections_silver",
  extra_configs = configs
)

True

In [0]:
df_byelection = spark.read.csv("/mnt/elections_raw/elections/TN Bye-Elections .csv", header=True, inferSchema=True)


In [0]:
display(df_byelection.limit(10))

AC Name,AC No,Type,State,Winning,Party,Electors,Votes,turnout,Reason,year
Panamarathupatty,91,NA,Tamil Nadu,K.Rajaram,ADK,NA,NA,NA,DEATH OF SHRI N.SUBBARAYAN,1977
Usilampatti,138,NA,Tamil Nadu,A.Thevar.S,FBL,NA,NA,NA,DEATH OF SHRI P.K.M.THAVER,1977
Vilavancode,233,NA,Tamil Nadu,Packiadas.D,CPM,NA,NA,NA,DEATH OF SHRI D.GNANASIGAMONY,1977
Anna,8,NA,Tamil Nadu,Eramachandran.S.M,DMK,NA,NA,NA,RESIGNATION OF Dr. M.KARUNANIDHI,1980
Mayuram,171,NA,Tamil Nadu,Sathiyaseelan.K,DMK,NA,NA,NA,DEATH OF SHRI N.KITTAPPA,1980
Thanjavur,183,NA,Tamil Nadu,A.Vandayar.K,INC,NA,NA,NA,DEATH OF SHRI N.NATARAJAN,1980
Tiruppattur,194,NA,Tamil Nadu,Arunagiri.R,INC(I),NA,NA,NA,DEATH OF SHRI V.VALMIGI,1980
Uppiliapuram,157,ST,Tamil Nadu,Saroja.R.(W),ADK,NA,NA,NA,VOID OF SHRI V.ARENGARAN ELECTION.,1980
Aruppukottai,204,NA,Tamil Nadu,V.S.Panchavarnam,ADK,NA,NA,NA,DEATH OF SHRI M.PITCHAI,1984
Cheyyar,50,NA,Tamil Nadu,V.Kuppusamy,ADK,NA,NA,NA,DEATH OF SHRI K.MURUGAN,1984


In [0]:
# Rename columns to clean, snake_case names
df_clean = (
    df_byelection
    .withColumnRenamed("AC Name", "ac_name")
    .withColumnRenamed("AC No", "ac_no")
    .withColumnRenamed("Type", "type")
    .withColumnRenamed("State", "state")
    .withColumnRenamed("Winning", "winning_cand")
    .withColumnRenamed("Party ", "party")   # removed trailing space
    .withColumnRenamed("Electors", "totelectors")
    .withColumnRenamed("Votes", "totvotes")
    .withColumnRenamed("turnout", "poll_percentage")
    .withColumnRenamed("Reason", "reason")
    .withColumnRenamed("year", "year")
)

# Save as Delta table
df_clean.write.format("delta").mode("overwrite").saveAsTable("tnelection.by_election")

In [0]:
%sql
select * from tnelection.by_election 
limit 10


ac_name,ac_no,type,state,winning_cand,party,totelectors,totvotes,poll_percentage,reason,year
Uppiliapuram,157,By election,Tamil Nadu,Saroja.R.(W),ADK,null,null,null,VOID OF SHRI V.ARENGARAN ELECTION.,1980
Natham,147,By election,Tamil Nadu,Viswanathan.R,All India Anna Dravida Munnetra Kazhagam,182655,113236,null,NA,1996
Tiruppattur,194,By election,Tamil Nadu,Arunagiri.R,Indian National Congress (I),null,null,null,DEATH OF SHRI V.VALMIGI,1980
Thanjavur,183,By election,Tamil Nadu,A.Vandayar.K,Indian National Congress,null,null,null,DEATH OF SHRI N.NATARAJAN,1980
Tiruppattur,41,By election,Tamil Nadu,"Manavalan,s.P",Indian National Congress,null,null,null,DEATH OF SHRI Y.SHANMUGAM,1984
Vilavancode,233,By election,Tamil Nadu,Packiadas.D,Communist Party Of India (MARXIST),null,null,null,DEATH OF SHRI D.GNANASIGAMONY,1977
Anna,8,By election,Tamil Nadu,Eramachandran.S.M,Dravida Munetra Kazhagam,null,null,null,RESIGNATION OF Dr. M.KARUNANIDHI,1980
Mayuram,171,By election,Tamil Nadu,Sathiyaseelan.K,Dravida Munetra Kazhagam,null,null,null,DEATH OF SHRI N.KITTAPPA,1980
Harbour,2,By election,Tamil Nadu,Selvarasam,Dravida Munetra Kazhagam,null,null,null,RESIGNATION OF SHRI M.KARUNIDHI,1991
Panamarathupatty,91,By election,Tamil Nadu,K.Rajaram,ADK,null,null,null,DEATH OF SHRI N.SUBBARAYAN,1977


In [0]:
%sql
-- SQL Script to Standardize Party Names in by_election table

-- 1. Communist Party of India (Marxist) (CPM)
UPDATE tnelection.by_election
SET party = 'Communist Party Of India (MARXIST)'
WHERE party = 'CPM';

-- 2. Dravida Munnetra Kazhagam (DMK)
UPDATE tnelection.by_election
SET party = 'Dravida Munetra Kazhagam'
WHERE party = 'DMK';

-- 3. Indian National Congress (INC and INC(I))
UPDATE tnelection.by_election
SET party = 'Indian National Congress'
WHERE party = 'INC';

UPDATE tnelection.by_election
SET party = 'Indian National Congress (I)'
WHERE party = 'INC(I)';

-- 4. All India Anna Dravida Munnetra Kazhagam (Various forms)
UPDATE tnelection.by_election
SET party = 'All India Anna Dravida Munnetra Kazhagam'
WHERE party IN ('ADMK', 'AIADMK', 'AIADK', 'AIDMK');

-- Standardize NA values to NULL
UPDATE tnelection.by_election
SET type = NULL
WHERE type = 'NA';

UPDATE tnelection.by_election
SET totelectors = NULL
WHERE totelectors = 'NA';

UPDATE tnelection.by_election
SET totvotes = NULL
WHERE totvotes = 'NA';

UPDATE tnelection.by_election
SET poll_percentage = NULL
WHERE poll_percentage = 'NA';

num_affected_rows
15


In [0]:
%sql
select * from tnelection.by_election
limit 10


ac_name,ac_no,type,state,winning_cand,party,totelectors,totvotes,poll_percentage,reason,year
Uppiliapuram,157,By election,Tamil Nadu,Saroja.R.(W),ADK,null,null,null,VOID OF SHRI V.ARENGARAN ELECTION.,1980
Natham,147,By election,Tamil Nadu,Viswanathan.R,All India Anna Dravida Munnetra Kazhagam,182655,113236,null,NA,1996
Tiruppattur,194,By election,Tamil Nadu,Arunagiri.R,Indian National Congress (I),null,null,null,DEATH OF SHRI V.VALMIGI,1980
Thanjavur,183,By election,Tamil Nadu,A.Vandayar.K,Indian National Congress,null,null,null,DEATH OF SHRI N.NATARAJAN,1980
Tiruppattur,41,By election,Tamil Nadu,"Manavalan,s.P",Indian National Congress,null,null,null,DEATH OF SHRI Y.SHANMUGAM,1984
Vilavancode,233,By election,Tamil Nadu,Packiadas.D,Communist Party Of India (MARXIST),null,null,null,DEATH OF SHRI D.GNANASIGAMONY,1977
Anna,8,By election,Tamil Nadu,Eramachandran.S.M,Dravida Munetra Kazhagam,null,null,null,RESIGNATION OF Dr. M.KARUNANIDHI,1980
Mayuram,171,By election,Tamil Nadu,Sathiyaseelan.K,Dravida Munetra Kazhagam,null,null,null,DEATH OF SHRI N.KITTAPPA,1980
Harbour,2,By election,Tamil Nadu,Selvarasam,Dravida Munetra Kazhagam,null,null,null,RESIGNATION OF SHRI M.KARUNIDHI,1991
Panamarathupatty,91,By election,Tamil Nadu,K.Rajaram,ADK,null,null,null,DEATH OF SHRI N.SUBBARAYAN,1977


In [0]:
%sql
-- Remove commas and cast to BIGINT
UPDATE tnelection.by_election
SET totelectors = CAST(regexp_replace(totelectors, ',', '') AS BIGINT);

UPDATE tnelection.by_election
SET totvotes = CAST(regexp_replace(totvotes, ',', '') AS BIGINT);

-- Remove % and cast to BIGINT
UPDATE tnelection.by_election
SET poll_percentage = CAST(
    CAST(regexp_replace(poll_percentage, '%', '') AS DOUBLE) AS BIGINT
);

num_affected_rows
34


In [0]:
%sql
select * from tnelection.by_election
limit 10

ac_name,ac_no,type,state,winning_cand,party,totelectors,totvotes,poll_percentage,reason,year
Uppiliapuram,157,By election,Tamil Nadu,Saroja.R.(W),ADK,null,null,null,VOID OF SHRI V.ARENGARAN ELECTION.,1980
Natham,147,By election,Tamil Nadu,Viswanathan.R,All India Anna Dravida Munnetra Kazhagam,182655,113236,null,NA,1996
Tiruppattur,194,By election,Tamil Nadu,Arunagiri.R,Indian National Congress (I),null,null,null,DEATH OF SHRI V.VALMIGI,1980
Thanjavur,183,By election,Tamil Nadu,A.Vandayar.K,Indian National Congress,null,null,null,DEATH OF SHRI N.NATARAJAN,1980
Tiruppattur,41,By election,Tamil Nadu,"Manavalan,s.P",Indian National Congress,null,null,null,DEATH OF SHRI Y.SHANMUGAM,1984
Vilavancode,233,By election,Tamil Nadu,Packiadas.D,Communist Party Of India (MARXIST),null,null,null,DEATH OF SHRI D.GNANASIGAMONY,1977
Anna,8,By election,Tamil Nadu,Eramachandran.S.M,Dravida Munetra Kazhagam,null,null,null,RESIGNATION OF Dr. M.KARUNANIDHI,1980
Mayuram,171,By election,Tamil Nadu,Sathiyaseelan.K,Dravida Munetra Kazhagam,null,null,null,DEATH OF SHRI N.KITTAPPA,1980
Harbour,2,By election,Tamil Nadu,Selvarasam,Dravida Munetra Kazhagam,null,null,null,RESIGNATION OF SHRI M.KARUNIDHI,1991
Panamarathupatty,91,By election,Tamil Nadu,K.Rajaram,ADK,null,null,null,DEATH OF SHRI N.SUBBARAYAN,1977


In [0]:
%sql
UPDATE tnelection.by_election
SET type = 'By election'


num_affected_rows
34


In [0]:
%sql
CREATE OR REPLACE VIEW tnelection.all_elections_view AS
SELECT 
    r.filename       AS year,
    r.ac_name,
    r.ac_no,
    r.type,
    r.district,
    r.winning_cand,
    r.party,
    r.totelectors,
    r.totvotes,
    r.poll_percentage,
    r.margin,
    r.winning_percentage
FROM tnelection.results r

UNION ALL

SELECT 
    b.year           AS year,
    b.ac_name,
    b.ac_no,
    b.type,
    NULL             AS district,         -- not available in by_election
    b.winning_cand,
    b.party,
    b.totelectors,
    b.totvotes,
    b.poll_percentage,
    NULL             AS margin,           -- not available in by_election
    NULL             AS winning_percentage -- not available in by_election
FROM tnelection.by_election b;

**Final Silver data**

In [0]:
%sql

SELECT *
FROM tnelection.all_elections_view
WHERE type <> 'By election'
   OR type IS NULL
limit 10;
    


year,ac_name,ac_no,type,district,winning_cand,party,totelectors,totvotes,poll_percentage,margin,winning_percentage
2006,Perambur,5,SC,null,"Mahendran, K.",Communist Party Of India (MARXIST),268704,182631,68,2788,2
2006,Gudiyatham,37,GEN,null,Latha.G,Communist Party Of India (MARXIST),178024,121384,68,1650,1
2006,Harur,79,SC,null,Dillibabu.P,Communist Party Of India (MARXIST),223534,155411,70,13693,9
2006,Tiruppur,116,GEN,null,Govindasamy.C,Communist Party Of India (MARXIST),428422,244208,57,11299,5
2006,Madurai East,144,GEN,null,Nanmaran.N,Communist Party Of India (MARXIST),134455,95253,71,51,0
2006,Dindigul,148,GEN,null,Balabarathi.K,Communist Party Of India (MARXIST),218221,152750,70,18949,12
2006,Nagapattinam,175,GEN,null,Marimuthu.V,Communist Party Of India (MARXIST),164819,125072,76,2344,2
2006,Thiruvattar,232,GEN,null,Leema Rose.R,Communist Party Of India (MARXIST),165801,110791,67,28050,25
2006,Vilavancode,233,GEN,null,"John Joseph,g",Communist Party Of India (MARXIST),195426,120076,61,45074,38
2011,Gummidipoondi,1,GEN,Thiruvallur,Sekar C H,Desiya Murpokku Dravida Kazhagam,215430,178276,83,29101,16


In [0]:
%sql

SELECT count(*)
FROM tnelection.all_elections_view 

count(1)
2840


**Loading silver data**

In [0]:
df_all = spark.sql("SELECT * FROM tnelection.all_elections_view")
df_all.write.format("delta").mode("overwrite").save("/mnt/election_silver")

In [0]:
df_final = spark.read.load("/mnt/election_silver")
display(df_final.limit(10))

year,ac_name,ac_no,type,district,winning_cand,party,totelectors,totvotes,poll_percentage,margin,winning_percentage
1980,Uppiliapuram,157,By election,null,Saroja.R.(W),ADK,null,null,null,null,null
1996,Natham,147,By election,null,Viswanathan.R,All India Anna Dravida Munnetra Kazhagam,182655,113236,null,null,null
1980,Tiruppattur,194,By election,null,Arunagiri.R,Indian National Congress (I),null,null,null,null,null
1980,Thanjavur,183,By election,null,A.Vandayar.K,Indian National Congress,null,null,null,null,null
1984,Tiruppattur,41,By election,null,"Manavalan,s.P",Indian National Congress,null,null,null,null,null
1977,Vilavancode,233,By election,null,Packiadas.D,Communist Party Of India (MARXIST),null,null,null,null,null
1980,Anna,8,By election,null,Eramachandran.S.M,Dravida Munetra Kazhagam,null,null,null,null,null
1980,Mayuram,171,By election,null,Sathiyaseelan.K,Dravida Munetra Kazhagam,null,null,null,null,null
1991,Harbour,2,By election,null,Selvarasam,Dravida Munetra Kazhagam,null,null,null,null,null
1977,Panamarathupatty,91,By election,null,K.Rajaram,ADK,null,null,null,null,null


**Final transformation & visualization using powerBI from silver**